In [248]:
import numpy as np
import pandas as pd
from itertools import combinations
from collections import Counter

In [249]:
df = pd.read_csv('movie_bd_v5.csv')
df.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
423,tt1212436,35000000,77477008,The Back-Up Plan,Jennifer Lopez|Alex O'Loughlin|Danneel Ackles|...,Alan Poul,"Fall in love, get married, have a baby. Not ne...","When Zoe tires of looking for Mr. Right, she d...",106,Comedy|Romance,Escape Artists|CBS Films,4/23/2010,5.7,2010
1045,tt1568338,42000000,46221189,Man on a Ledge,Sam Worthington|Elizabeth Banks|Jamie Bell|Ed ...,Asger Leth,You can only push an innocent man so far,An ex-cop turned con threatens to jump to his ...,102,Action|Thriller|Crime,Summit Entertainment|Di Bonaventura Picture,1/13/2012,6.2,2012
1647,tt0327437,110000000,72178895,Around the World in 80 Days,Jackie Chan|Steve Coogan|CÃ©cile De France|Rob...,Frank Coraci,Let your imagination soar.,"A bet pits a British inventor, a Chinese thief...",120,Action|Adventure|Comedy,Studio Babelsberg|Walden Media|Babelsberg Film...,6/16/2004,5.6,2004
1488,tt0421729,40000000,138259062,Big Momma's House 2,Martin Lawrence|Nia Long|Emily Procter|Zachary...,John Whitesell,The momma of all comedies is back!!!,FBI agent Malcolm Turner goes back undercover ...,99,Comedy|Crime,Twentieth Century Fox Film Corporation|Regency...,1/26/2006,5.6,2006
419,tt1038919,40000000,136000000,The Bounty Hunter,Gerard Butler|Jennifer Aniston|Gio Perez|Siobh...,Andy Tennant,"It's a Job. It Isn't Personal. Well, Maybe a L...",Milo Boyd is a bounty hunter whose latest gig ...,110,Action,Columbia Pictures|Original Film|Relativity Med...,3/16/2010,5.5,2010


In [250]:
df.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [251]:
answers = {}


# some questions required season grouping
def get_season_by_month(month):
    if month in [12, 1, 2]:
        return 'winter'
    if month in [3, 4, 5]:
        return 'spring'
    if month in [6, 7, 8]:
        return 'summer'
    if month in [9, 10, 11]:
        return 'fall'
    raise RuntimeError('invalid month provided: ' + month)


# to group by actor combinations we need to split cast list and sort it
def split_sorted(string):
    result = string.split('|')
    result.sort()
    return result


df['full_title'] = df['original_title'] + ' (' + df['imdb_id'] + ')'
df['original_title_len'] = df['original_title'].apply(lambda s: len(s))
# profit = revenue - budget
df['profit'] = df['revenue'] - df['budget']
df['genres_split'] = df['genres'].apply(lambda x: x.split('|'))
df['director_split'] = df['director'].apply(lambda x: x.split('|'))
# use split_sorted in order to make actors combinations similar
df['cast_split'] = df['cast'].apply(split_sorted)
df['cast_split_combinations'] = df['cast_split'].apply(lambda x: list(combinations(x, 2)))
df['prod_companies_split'] = df['production_companies'].apply(lambda x: x.split('|'))
df['release_month'] = df['release_date'].apply(lambda x: int(x.split('/')[0]))
df['release_season'] = df['release_month'].apply(get_season_by_month)
df['overview_words_count'] = df['overview'].apply(lambda s: len(s.split()))

df.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,original_title_len,profit,genres_split,director_split,cast_split,cast_split_combinations,prod_companies_split,release_month,release_season,overview_words_count
1623,tt0375173,60000000,13395939,Alfie,Jude Law|Marisa Tomei|Omar Epps|Jane Krakowski...,Charles Shyer,Meet a man who never met a woman he didn't love.,"In Manhattan, the British limousine driver Alf...",103,Comedy,...,5,-46604061,[Comedy],[Charles Shyer],"[Jane Krakowski, Jude Law, Marisa Tomei, Omar ...","[(Jane Krakowski, Jude Law), (Jane Krakowski, ...","[Paramount Pictures, Patalex Productions]",10,fall,57
1238,tt1245492,32000000,126041322,This Is the End,James Franco|Jonah Hill|Seth Rogen|Jay Baruche...,Seth Rogen|Evan Goldberg,Nothing ruins a party like the end of the world.,While attending a party at James Franco's hous...,107,Action|Comedy,...,15,94041322,"[Action, Comedy]","[Seth Rogen, Evan Goldberg]","[Danny McBride, James Franco, Jay Baruchel, Jo...","[(Danny McBride, James Franco), (Danny McBride...","[Columbia Pictures, Mandate Pictures, Sony Pic...",6,summer,21
694,tt1031969,15000000,8762890,The Rocker,Rainn Wilson|Josh Gad|Christina Applegate|Emma...,Peter Cattaneo,Opportunity rocks.,"Rob ""Fish"" Fishman is the drummer in '80s hair...",102,Comedy|Music,...,10,-6237110,"[Comedy, Music]",[Peter Cattaneo],"[Christina Applegate, Emma Stone, Josh Gad, Ra...","[(Christina Applegate, Emma Stone), (Christina...",[Fox Atomic],8,summer,43
1490,tt0424345,5000000,26888376,Clerks II,Brian O'Halloran|Jeff Anderson|Jason Mewes|Kev...,Kevin Smith,With No Power Comes No Responsibility,A calamity at Dante and Randall's shops sends ...,97,Comedy,...,9,21888376,[Comedy],[Kevin Smith],"[Brian O'Halloran, Jason Mewes, Jeff Anderson,...","[(Brian O'Halloran, Jason Mewes), (Brian O'Hal...","[The Weinstein Company, View Askew Productions]",5,spring,25
75,tt4178092,5000000,58978653,The Gift,Jason Bateman|Rebecca Hall|Joel Edgerton|David...,Joel Edgerton,Not every gift is welcome.,A husband and wife try to reinvigorate their r...,108,Thriller|Mystery,...,8,53978653,"[Thriller, Mystery]",[Joel Edgerton],"[Beau Knapp, David Denman, Jason Bateman, Joel...","[(Beau Knapp, David Denman), (Beau Knapp, Jaso...","[Blumhouse Productions, Blue-Tongue Films, STX...",7,summer,34


In [252]:
genres_split_df = df.explode('genres_split')
prod_companies_split_df = df.explode('prod_companies_split')
director_split_df = df.explode('director_split')
director_genres_split_df = director_split_df.explode('genres_split')
cast_split_df = df.explode('cast_split')
cast_genres_split_df = cast_split_df.explode('genres_split')
cast_split_combinations_df = df.explode('cast_split_combinations')

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [253]:
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

In [254]:
df[df['budget'] == max(df['budget'])]['full_title'].iloc[0]

'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

# 2. Какой из фильмов самый длительный (в минутах)?

In [255]:
answers['2'] = 'Gods and Generals (tt0279111)'

In [256]:
df[df['runtime'] == max(df['runtime'])]['full_title'].iloc[0]

'Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?

In [257]:
answers['3'] = 'Winnie the Pooh (tt1449283)'

In [258]:
df[df['runtime'] == min(df['runtime'])]['full_title'].iloc[0]

'Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?

In [259]:
answers['4'] = 110

In [260]:
np.mean(df['runtime'])

109.6585494970884

# 5. Каково медианное значение длительности фильмов? 

In [261]:
answers['5'] = 107.0

In [262]:
np.median(df['runtime'])

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [263]:
answers['6'] = 'Avatar (tt0499549)'

In [264]:
df[df['profit'] == max(df['profit'])]['full_title'].iloc[0]

'Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [265]:
answers['7'] = 'The Lone Ranger (tt1210819)'

In [266]:
df[df['profit'] == min(df['profit'])]['full_title'].iloc[0]

'The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [267]:
answers['8'] = 1478

In [268]:
len(df[df['revenue'] > df['budget']])

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [269]:
answers['9'] = 'The Dark Knight (tt0468569)'

In [270]:
release_2008_df = df[(df['release_year'] == 2008)]
release_2008_max_profit_df = release_2008_df[release_2008_df['profit'] == max(release_2008_df['profit'])]
release_2008_max_profit_df['full_title'].iloc[0]

'The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [271]:
answers['10'] = 'The Lone Ranger (tt1210819)'

In [272]:
release_2012_2014_df = df[(df['release_year'] >= 2012) & (df['release_year'] <= 2014)]
min_profit_df = release_2012_2014_df[release_2012_2014_df['profit'] == min(release_2012_2014_df['profit'])]
min_profit_df['full_title'].iloc[0]

'The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [273]:
answers['11'] = 'Drama'

ВАРИАНТ 1

In [274]:
Counter(df['genres'].str.cat(sep='|').split('|')).most_common()[0][0]

'Drama'

ВАРИАНТ 2

In [275]:
Counter(genres_split_df['genres_split']).most_common()[0][0]

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [276]:
answers['12'] = 'Drama'

ВАРИАНТ 1

In [277]:
profitable_df = df[df['profit'] > 0]
Counter(profitable_df['genres'].str.cat(sep='|').split('|')).most_common()[0][0]

'Drama'

ВАРИАНТ 2

In [278]:
profitable_df = genres_split_df[genres_split_df['profit'] > 0]
Counter(profitable_df['genres_split']).most_common()[0][0]

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [279]:
answers['13'] = 'Peter Jackson'

In [280]:
revenue_df = director_split_df.groupby('director_split')['revenue']
revenue_df.apply(sum).sort_values(ascending=False).keys()[0]

'Peter Jackson'

# 14. Какой режиссер снял больше всего фильмов в стиле Action?

In [281]:
answers['14'] = 'Robert Rodriguez'

In [282]:
action_df = director_genres_split_df[director_genres_split_df['genres_split'] == 'Action']
action_df['director_split'].value_counts().sort_values(ascending=False).keys()[0]

'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [283]:
answers['15'] = 'Chris Hemsworth'

In [284]:
release_2012_df = cast_split_df[cast_split_df['release_year'] == 2012]
revenue_df = release_2012_df.groupby('cast_split')['revenue']
revenue_df.apply(sum).sort_values(ascending=False).keys()[0]

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [285]:
answers['16'] = 'Matt Damon'

In [286]:
high_budget_df = cast_split_df[cast_split_df['budget'] >= np.mean(cast_split_df['budget'])]
high_budget_df['cast_split'].value_counts().sort_values(ascending=False).keys()[0]

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [287]:
answers['17'] = 'Action'

In [288]:
ncage_df = cast_genres_split_df[cast_genres_split_df['cast_split'] == 'Nicolas Cage']
ncage_df['genres_split'].value_counts().sort_values(ascending=False).keys()[0]

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [289]:
answers['18'] = 'K-19: The Widowmaker (tt0267626)'

In [290]:
para_df = prod_companies_split_df[prod_companies_split_df['prod_companies_split'] == 'Paramount Pictures']
para_df[para_df['profit'] == min(para_df['profit'])]['full_title'].iloc[0]

'K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [291]:
answers['19'] = 2015

In [292]:
df.groupby('release_year')['revenue'].apply(sum).sort_values(ascending=False).keys()[0]

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [293]:
answers['20'] = 2014

In [294]:
warner_df = prod_companies_split_df[prod_companies_split_df['prod_companies_split'].str.startswith('Warner Bros', na=False)]
warner_df.groupby('release_year')['profit'].apply(sum).sort_values(ascending=False).keys()[0]

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [295]:
answers['21'] = 'Сентябрь'

In [296]:
df['release_month'].value_counts().sort_values(ascending=False).keys()[0]

9

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [297]:
answers['22'] = 450

In [298]:
df['release_season'].value_counts()['summer']

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [299]:
answers['23'] = 'Peter Jackson'

In [300]:
winter_df = director_split_df[director_split_df['release_season'] == 'winter']['director_split']
winter_df.value_counts().sort_values(ascending=False).keys()[0]

'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [301]:
answers['24'] = 'Four By Two Productions'

In [302]:
title_len_df = prod_companies_split_df.groupby('prod_companies_split')['original_title_len']
title_len_df.apply(np.mean).sort_values(ascending=False).keys()[0]

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [303]:
answers['25'] = 'Midnight Picture Show'

In [304]:
overview_words_count_df = prod_companies_split_df.groupby('prod_companies_split')['overview_words_count']
overview_words_count_df.apply(np.mean).sort_values(ascending=False).keys()[0]

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [305]:
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'

In [306]:
df[df['vote_average'] >= df['vote_average'].quantile(0.99)]['original_title']

9                                              Inside Out
28                                              Spotlight
34                                                   Room
118                                          Interstellar
119                               Guardians of the Galaxy
124                                            Big Hero 6
125                                    The Imitation Game
128                                             Gone Girl
138                              The Grand Budapest Hotel
155                              The Theory of Everything
177                                The Fault in Our Stars
283                                            Mr. Nobody
316                                              3 Idiots
370                                             Inception
496     The Lord of the Rings: The Fellowship of the Ring
599                                       The Dark Knight
863                 The Lord of the Rings: The Two Towers
872           

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [307]:
answers['27'] = 'Daniel Radcliffe & Rupert Grint'

In [308]:
cast_combinations_df = cast_split_combinations_df.groupby('cast_split_combinations')['cast_split_combinations']
cast_combinations_df.agg(['count']).sort_values(by='count',ascending=False)

,count
cast_split_combinations,
"(Daniel Radcliffe, Rupert Grint)",8
"(Emma Watson, Rupert Grint)",8
"(Daniel Radcliffe, Emma Watson)",8
"(Ben Stiller, Owen Wilson)",6
"(Helena Bonham Carter, Johnny Depp)",6
...,...
"(Christopher Walken, Robin Williams)",1
"(Christopher Walken, Rosario Dawson)",1
"(Christopher Walken, Sam Rockwell)",1


# Submission

In [309]:
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107.0,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': 'Сентябрь',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [310]:
len(answers)

27